In [ ]:
# Imports - DATA PROCESSING
# Task 1 - Build Classifier using ResNet-18
# Task 2 - Add adversarial Debiasing with GRL
# Task 3 - Evaluate Fairness Improvents (accuracy on l, m, d separately), absolute accuracy gaps, Grad-CAM focus changes
# Task 4 - Grad-CAM visualizations with basline and debiased model
# -- we are looking for basline to focus background lighting or arm position, and debiased to focus more on hand shape only.


In [ ]:
# MOUNT THE DRIVE - FOR COLAB
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
#data_dir = "/content/drive/MyDrive/Colab\Notebooks/data/asldata/root"

In [ ]:
!ls "asldata"

In [ ]:
#!rm -rf data
!unzip "asldata/Root.zip" -d "data"

In [ ]:
#suvethika
!ls "data/Images of American Sign Language (ASL) Alphabet Gestures"

In [ ]:
#suvethika
!unzip "data/Images of American Sign Language (ASL) Alphabet Gestures/Root.zip" -d "data/asl"

In [ ]:
"""
# =========================================================
# FUNCTION TO MANUALLY LABEL DATA - RUN ONCE
# =========================================================

# =========================================================
# CONFIG
# =========================================================
TRAIN_ROOT = "data/train"     # or your full train path
SAMPLES_PER_LETTER = 18
CSV_PATH = "data/asl_manual_labels.csv"


# =========================================================
# 1. BUILD / LOAD SAMPLE DF
# =========================================================
def make_sample_df(train_root=TRAIN_ROOT, samples_per_letter=SAMPLES_PER_LETTER, seed=42):
    random.seed(seed)
    rows = []

    for letter in sorted(os.listdir(train_root)):
        letter_dir = os.path.join(train_root, letter)
        if not os.path.isdir(letter_dir):
            continue

        imgs = [
            os.path.join(letter_dir, f)
            for f in os.listdir(letter_dir)
            if f.lower().endswith(".jpg")
        ]
        if not imgs:
            continue

        chosen = random.sample(imgs, min(samples_per_letter, len(imgs)))
        for path in chosen:
            rows.append({"filepath": path, "letter": letter, "skin_tone": None})

    df = pd.DataFrame(rows)
    # shuffle so you don't do all A's then all B's
    df = df.sample(frac=1, random_state=seed).reset_index(drop=True)
    return df

# create or load
if os.path.exists(CSV_PATH):
    df = pd.read_csv(CSV_PATH)
    print(f"Loaded existing labels from {CSV_PATH}")
else:
    df = make_sample_df()
    df.to_csv(CSV_PATH, index=False)
    print(f"Created new sample and saved to {CSV_PATH}")

# start at first unlabeled index
unlabeled_idx = df["skin_tone"].isna()
if unlabeled_idx.any():
    index = unlabeled_idx.idxmax()
else:
    index = 0

print(f"Starting at index {index} of {len(df)} total images.")


# =========================================================
# 2. IMAGE DISPLAY AREA
# =========================================================
image_out = widgets.Output()

def show_image(i):
    image_out.clear_output(wait=True)
    with image_out:
        if i >= len(df):
            print("🎉 All images labeled!")
            return

        row = df.iloc[i]
        img = Image.open(row.filepath).convert("RGB")
        plt.figure(figsize=(3, 3))
        plt.imshow(img)
        plt.axis("off")
        plt.title(f"Index {i} | Letter: {row.letter}")
        plt.show()


# =========================================================
# 3. TEXT INPUT FOR KEYBOARD LABELS
# =========================================================
label_box = widgets.Text(
    value='',
    placeholder='l = light, m = medium, d = dark, s = skip, q = quit',
    description='Label:',
    disabled=False
)

status_out = widgets.Output()

def handle_label_submission(change):
    global index, df
    text = change.value.strip().lower()
    label_box.value = ''  # clear box for next input

    if index >= len(df):
        with status_out:
            status_out.clear_output(wait=True)
            print("All images already labeled.")
        return

    if text in ['q', 'quit']:
        with status_out:
            status_out.clear_output(wait=True)
            print(f"Stopped labeling at index {index}. Progress saved to {CSV_PATH}.")
        return

    if text in ['l', 'light']:
        tone = 'light'
    elif text in ['m', 'medium']:
        tone = 'medium'
    elif text in ['d', 'dark']:
        tone = 'dark'
    elif text in ['s', 'skip', '']:
        tone = None
    else:
        # invalid input; show help
        with status_out:
            status_out.clear_output(wait=True)
            print("Invalid input. Use l/m/d/s/q.")
        return

    if tone is not None:
        df.at[index, "skin_tone"] = tone
        df.to_csv(CSV_PATH, index=False)
        with status_out:
            status_out.clear_output(wait=True)
            print(f"Saved index={index} → {tone}")
    else:
        with status_out:
            status_out.clear_output(wait=True)
            print(f"Skipped index={index}")

    index += 1
    show_image(index)

label_box.on_submit(handle_label_submission)


# =========================================================
# 4. LAYOUT & START
# =========================================================
ui = widgets.VBox([
    image_out,
    label_box,
    status_out
])

display(ui)
show_image(index)
"""


In [ ]:
# Check CSV exists
CSV_PATH = "data/asl_manual_labels.csv"
print("Exists?", os.path.exists(CSV_PATH))


In [ ]:
# MARK MPS FOR MACBOOK 
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
device


In [ ]:
MANUAL_CSV = "data/asl_manual_labels.csv"  # path to your labeled subset

df_manual = pd.read_csv(MANUAL_CSV)
print(df_manual.head())
print(df_manual["skin_tone"].value_counts())


In [ ]:
# encode skin tone labels
skin_tone_to_id = {"light": 0, "medium": 1, "dark": 2}
id_to_skin_tone = {v: k for k, v in skin_tone_to_id.items()}

df_manual = df_manual[df_manual["skin_tone"].notna()].copy()
df_manual["skin_tone_id"] = df_manual["skin_tone"].map(skin_tone_to_id)
df_manual["skin_tone_id"].value_counts()


In [ ]:
# define dataset class for skin tone classification
transform = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225]),
])

class SkinToneDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df.reset_index(drop=True)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img = Image.open(row["filepath"]).convert("RGB")
        if self.transform is not None:
            img = self.transform(img)
        label = int(row["skin_tone_id"])
        return img, label


In [ ]:
# create dataset and dataloaders
full_dataset = SkinToneDataset(df_manual, transform=transform)

val_frac = 0.2
n_total = len(full_dataset)
n_val = int(n_total * val_frac)
n_train = n_total - n_val

train_ds, val_ds = random_split(full_dataset, [n_train, n_val])

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True, num_workers=0)
val_loader = DataLoader(val_ds, batch_size=64, shuffle=False, num_workers=0)

n_train, n_val


In [ ]:
# define model, loss, optimizer
num_classes = 3  # light, medium, dark

model = models.mobilenet_v3_small(weights=models.MobileNet_V3_Small_Weights.DEFAULT)
# Replace classifier head
in_features = model.classifier[-1].in_features
model.classifier[-1] = nn.Linear(in_features, num_classes)

model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)


In [ ]:
BEST_MODEL_PATH = "skin_tone_mobilenet_best.pth" # path to save best model for skin tone classification

In [ ]:
# =========================================================
# RUN BEST MODEL TO LABEL 87000 IMAGES
# =========================================================


def train_one_epoch(model, loader, optimizer, criterion):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, preds = outputs.max(1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    avg_loss = running_loss / total
    acc = correct / total
    return avg_loss, acc

def eval_model(model, loader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * images.size(0)
            _, preds = outputs.max(1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    avg_loss = running_loss / total
    acc = correct / total
    return avg_loss, acc

num_epochs = 8  # you can tweak this
best_val_acc = 0.0

for epoch in range(num_epochs):
    train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, criterion)
    val_loss, val_acc = eval_model(model, val_loader, criterion)

    print(f"Epoch {epoch+1}/{num_epochs} "
          f"Train loss: {train_loss:.4f}, acc: {train_acc:.3f} | "
          f"Val loss: {val_loss:.4f}, acc: {val_acc:.3f}")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), BEST_MODEL_PATH)
        print(f"  🔥 New best model saved with val_acc={val_acc:.3f}")


In [ ]:
model.load_state_dict(torch.load(BEST_MODEL_PATH, map_location=device))
model.eval()
best_val_acc


In [ ]:
TRAIN_ROOT = "data/train"  # change if your path is different

all_rows = []

for letter in sorted(os.listdir(TRAIN_ROOT)):
    letter_dir = os.path.join(TRAIN_ROOT, letter)
    if not os.path.isdir(letter_dir):
        continue

    for fname in os.listdir(letter_dir):
        if fname.lower().endswith(".jpg"):
            fpath = os.path.join(letter_dir, fname)
            all_rows.append({"filepath": fpath, "letter": letter})

df_all = pd.DataFrame(all_rows)
print("Total images:", len(df_all))
df_all.head()


In [ ]:
class AllImagesDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df.reset_index(drop=True)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img = Image.open(row["filepath"]).convert("RGB")
        if self.transform is not None:
            img = self.transform(img)
        return img, row["filepath"]

all_dataset = AllImagesDataset(df_all, transform=transform)
all_loader = DataLoader(all_dataset, batch_size=64, shuffle=False, num_workers=0)

pred_skin_ids = []
pred_paths = []

model.eval()
with torch.no_grad():
    for images, paths in tqdm(all_loader, desc="Predicting skin tones"):
        images = images.to(device)
        outputs = model(images)
        _, preds = outputs.max(1)
        preds = preds.cpu().numpy().tolist()

        pred_skin_ids.extend(preds)
        pred_paths.extend(list(paths))

len(pred_skin_ids), len(pred_paths)


In [ ]:
df_pred = pd.DataFrame({
    "filepath": pred_paths,
    "skin_tone_id_pred": pred_skin_ids
})
df_pred["skin_tone_pred"] = df_pred["skin_tone_id_pred"].map(id_to_skin_tone)

df_pred.head()


In [ ]:
# =========================================================
# 3. merge all the data
# =========================================================
# 1) Make sure we have fresh copies
df_all = df_all.copy()       # ['filepath', 'letter']
df_pred = df_pred.copy()     # ['filepath', 'skin_tone_id_pred', 'skin_tone_pred']
df_manual = df_manual.copy() # ['filepath', 'letter', 'skin_tone', 'skin_tone_id']

# 2) Keep only the manual label columns we need (avoid letter conflict)
df_manual_small = df_manual[["filepath", "skin_tone", "skin_tone_id"]].copy()

# 3) Rename manual columns so we can distinguish them
df_manual_small = df_manual_small.rename(columns={
    "skin_tone": "skin_tone_manual",
    "skin_tone_id": "skin_tone_id_manual"
})

# 4) First merge: all images + predicted labels
df_merged = df_all.merge(df_pred, on="filepath", how="left")

# 5) Second merge: add manual labels where available
df_merged = df_merged.merge(df_manual_small, on="filepath", how="left")

# 6) Create final label columns:
#    - if a manual label exists, use that
#    - otherwise use the predicted label
df_merged["skin_tone_final"] = df_merged["skin_tone_manual"].combine_first(
    df_merged["skin_tone_pred"]
)

df_merged["skin_tone_id_final"] = df_merged["skin_tone_id_manual"].combine_first(
    df_merged["skin_tone_id_pred"]
)

# 7) Quick sanity checks
print(df_merged[["filepath", "letter", "skin_tone_pred", "skin_tone_manual", "skin_tone_final"]].head())
print("\nCounts of final labels:")
print(df_merged["skin_tone_final"].value_counts(dropna=False))

# 8) Save out final CSV
OUT_CSV = "data/asl_all_with_skin_tones.csv"
df_merged.to_csv(OUT_CSV, index=False)
print("\nSaved:", OUT_CSV, "with", len(df_merged), "rows")

In [ ]:
# CLASSIFIER USING RESNET-18

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
device


In [ ]:
# see if CSV_PATH = "data/asl_all_with_skin_tones.csv" exists
CSV_PATH = "data/asl_all_with_skin_tones.csv"
print("Exists?", os.path.exists(CSV_PATH))

#why is it empty?
df_all_skin = pd.read_csv(CSV_PATH)
print(df_all_skin.head())
print(df_all_skin["skin_tone_final"].value_counts())


In [ ]:
CSV_PATH = "data/asl_all_with_skin_tones.csv"  # adjust if needed

df = pd.read_csv(CSV_PATH)

# We only need these columns for baseline
df = df[["filepath", "letter"]].dropna()

# Make sure paths exist (optional but nice sanity check)
df = df[df["filepath"].apply(os.path.exists)]
print("Total images after path check:", len(df))

# Map letters to integer IDs
letters = sorted(df["letter"].unique())
letter_to_id = {letter: idx for idx, letter in enumerate(letters)}
id_to_letter = {v: k for k, v in letter_to_id.items()}

df["label_id"] = df["letter"].map(letter_to_id)

print("Num classes:", len(letters))
print("Classes:", letters)
